In [1]:
# save processed speech command data as wav file
save_processed_speech_cmd_files = False
# Use data augmentation
use_augmentation = True
# whether to train on whole dataset 
TRAIN_RAKSHAK_ON_ALL_DATA = True

In [2]:
TRAIN = 'rakshak'
TEST = ['speech_cmd', 'rakshak']

In [3]:
speech_path = "../input/train/audio/"
speech_files = ['go', 'stop', 'yes', 'no']
background_noises = ['dude_miaowing.wav','pink_noise.wav','white_noise.wav','doing_the_dishes.wav','exercise_bike.wav','running_tap.wav']
speech_setup = 'speech_setup/'
speech_setup_data = speech_path+'data/'
rakshak_path = 'rakshak/'

In [4]:
!pip install kapre tensorflow-gpu

     |████████████████████████████████| 377.0MB 49kB/s 


In [5]:
# put dataset in folder named rakshak

Cloning into 'rakshak'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (335/335), done.
remote: Total 5676 (delta 9), reused 328 (delta 2), pack-reused 5339
Receiving objects: 100% (5676/5676), 406.24 MiB | 20.73 MiB/s, done.
Resolving deltas: 100% (415/415), done.
Checking out files: 100% (4994/4994), done.


In [6]:
import os
import random
import numpy as np
from scipy.io import wavfile
import librosa
from tqdm import tqdm
import gc
random.seed(4)
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from matplotlib.backend_bases import RendererBase

import plotly as py
py.tools.set_credentials_file(username='Ujjwal999',api_key='QNq0B1RHjBetoHBqLOjy')

import plotly.plotly as py
import plotly.graph_objs as go

import cufflinks as cf

In [7]:
if save_processed_speech_cmd_files:
    if not os.path.isdir(speech_setup_data):
        os.mkdir(speech_setup_data)
    for dir_ in speech_files: 
        if not os.path.isdir(speech_setup_data+dir_):
            os.mkdir(speech_setup_data+dir_)

In [8]:
def roll(data):
    data_roll = np.roll(data, 5000)
    return data_roll

def stretch(data, rate=2):
    input_length = 16000*3
    data = librosa.effects.time_stretch(np.array(data, dtype=np.float32), rate)
    if len(data)>input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
    return data.tolist()

def wnoise(data):
    wn = np.random.randn(len(data))
    data_wn = data + 0.005*wn
    return data_wn

In [9]:
def make_3_second(path, files):
    r = random.random()
    sr, data = wavfile.read(path+files)
    data = data.tolist()
    if r < 0.3:
        background_files = path + '_background_noise_/'
        choice = random.choice(background_noises)
        sr_1, data_1 = wavfile.read(background_files + choice)
        data.extend(data_1)
    else:
        data_1 = [0]*32000
        data.extend(data_1)
        data = data[:48000]
        if len(data) != 48000:
            data.extend([0]*(48000-len(data)))
    return data

In [10]:
def speech_command(new_split=True, speech_setup=speech_setup):
    speech_split = {'train.txt':[], 'test.txt':[], 'valid.txt':[]}
    for file in speech_split.keys():
        if os.path.exists(speech_setup+file):
#             os.remove(speech_setup+file)
#             split_speech_cmd = True
            new_split = False
            with open(file) as f:
                for line in f:
                    speech_split[key].append(line.strip())
    speech_cmd = {'train':[[],[]], 'valid':[[],[]], 'test':[[],[]]}
    to_test = True
    for folder in speech_files:
        files = os.listdir(speech_path+folder)
        for file in tqdm(files):
            file_path = folder+'/'+file
            if new_split == False and os.path.isdir(speech_setup_data):
                sr, data = wavfile.read(speech_path, file_path)
#             if save_processed_speech_cmd_files and new_split:
#                 wavfile.write(speech_setup_data+file_path, 16000, np.array(data, dtype=np.int16))
            if new_split:
                r = random.random()
                if r <= 0.7:
                    if TRAIN == 'speech_cmd':
                        data = make_3_second(speech_path, file_path)
                        speech_split['train.txt'].append(file_path)
                        speech_cmd['train'][0].append(data[:48000])
                        speech_cmd['train'][1].append(folder)
                    else:
                        continue
                else:
                    data = make_3_second(speech_path, file_path)
                    if to_test:
                        to_test = False
                        speech_split['test.txt'].append(file_path)
                        speech_cmd['test'][0].append(data[:48000])
                        speech_cmd['test'][1].append(folder)
                    else:
                        to_test = True
                        speech_split['valid.txt'].append(file_path)
                        speech_cmd['valid'][0].append(data[:48000])
                        speech_cmd['valid'][1].append(folder)
            else:
                if (file_path in speech_split['train.txt']):
                    if TRAIN == 'speech_cmd':
                        speech_cmd['train'][0].append(data[:48000])
                        speech_cmd['train'][1].append(folder)
                elif file_path in speech_split['valid.txt']:
                    speech_cmd['valid'][0].append(data)
                    speech_cmd['valid'][1].append(folder)
                else:
                    speech_cmd['test'][0].append(data)
                    speech_cmd['test'][1].append(folder)
    return speech_split, speech_cmd

In [11]:
speech_split, speech_cmd = speech_command()
gc.collect()

100%|██████████| 2375/2375 [00:18<00:00, 131.81it/s]


0

In [12]:
for key in speech_cmd.keys():
    speech_cmd[key][0] = np.array(speech_cmd[key][0])
    speech_cmd[key][1] = np.array(speech_cmd[key][1])

In [13]:
for key in speech_cmd.keys():
    print(key, ":", speech_cmd[key][0].shape)

train : (0,)
valid : (1445, 48000)
test : (1446, 48000)


In [14]:
for file in speech_split.keys():
    with open('speech_'+file, 'w') as f:
        for item in speech_split[file]:
            f.write("%s\n" % item)
gc.collect()

0

In [15]:
def rakshak_dataset(new_split=True, rakshak_path=rakshak_path, datatype='keyword'):
    rakshak_split = {'rakshak_train.txt':[], 'rakshak_test.txt':[], 'rakshak_valid.txt':[]}
    for file in rakshak_split.keys():
        if os.path.exists(rakshak_path+file):
#             os.remove(datapath+file)
#             new_split = True
            new_split = False
            with open(file) as f:
                for line in f:
                    rakshak_split[key].append(line.strip())
    rakshak = {'train':[[],[]], 'valid':[[],[]], 'test':[[],[]]}
    to_test = True
    idx = {'keyword':0, 'background':1, 'stress':2}
    datapath = rakshak_path + 'paperdata/'
    for folder in os.listdir(datapath):
        files = os.listdir(datapath+folder)
        for file in tqdm(files):
            file_path = folder+'/'+file
            sr, data = wavfile.read(datapath+file_path)
            if data.shape[0] < 48000:
                data = np.append(data, [0]*(48000-data.shape[0]))
            if new_split:
                r = random.random()
                if file.split('-')[idx[datatype]] in speech_files:
                    if r <= 0.7:
                        if TRAIN != 'rakshak':
                            continue
                        rakshak_split['rakshak_train.txt'].append(file_path)
                        rakshak['train'][0].append(data[:48000])
                        rakshak['train'][1].append(file.split('-')[idx[datatype]])
                        if use_augmentation:
                            rakshak['train'][0].append(roll(data)[:48000])
                            rakshak['train'][1].append(file.split('-')[idx[datatype]])
                            rakshak['train'][0].append(stretch(data)[:48000])
                            rakshak['train'][1].append(file.split('-')[idx[datatype]])
                            rakshak['train'][0].append(wnoise(data)[:48000])
                            rakshak['train'][1].append(file.split('-')[idx[datatype]])
                    else:
                        if to_test:
                            to_test = False
                            rakshak_split['rakshak_test.txt'].append(file_path)
                            rakshak['test'][0].append(data[:48000])
                            rakshak['test'][1].append(file.split('-')[idx[datatype]])
                        else:
                            to_test = True
                            rakshak_split['rakshak_valid.txt'].append(file_path)
                            rakshak['valid'][0].append(data[:48000])
                            rakshak['valid'][1].append(file.split('-')[idx[datatype]])
            else:
                if file_path in rakshak_split['rakshak_train.txt']:
                    if TRAIN != 'rakshak':
                        rakshak['train'][0].append(data[:48000])
                        rakshak['train'][1].append(file.split('-')[idx[datatype]])
                        if use_augmentation:
                            rakshak['train'][0].append(roll(data)[:48000])
                            rakshak['train'][1].append(file.split('-')[idx[datatype]])
                            rakshak['train'][0].append(stretch(data)[:48000])
                            rakshak['train'][1].append(file.split('-')[idx[datatype]])
                            rakshak['train'][0].append(wnoise(data)[:48000])
                            rakshak['train'][1].append(file.split('-')[idx[datatype]])
                elif file_path in rakshak_split['rakshak_valid.txt']:
                    rakshak['valid'][0].append(data[:48000])
                    rakshak['valid'][1].append(file.split('-')[idx[datatype]])
                elif file_path in rakshak_split['rakshak_test.txt']:
                    rakshak['test'][0].append(data[:48000])
                    rakshak['test'][1].append(file.split('-')[idx[datatype]])
    return rakshak_split, rakshak

In [16]:
rakshak_split, rakshak = rakshak_dataset()
gc.collect()

100%|██████████| 73/73 [00:00<00:00, 1538.51it/s]


21411

In [17]:
for key in rakshak.keys():
    rakshak[key][0] = np.array(rakshak[key][0])
    rakshak[key][1] = np.array(rakshak[key][1])

In [18]:
for key in rakshak.keys():
    print(key, ":", rakshak[key][0].shape)

train : (2440, 48000)
valid : (121, 48000)
test : (122, 48000)


In [19]:
if TRAIN_RAKSHAK_ON_ALL_DATA:
    all_rakshak_data = np.vstack((rakshak['train'][0], rakshak['test'][0]))
    all_rakshak_labels = []
    for key in rakshak.keys():
        if key == 'valid':
            continue
        all_rakshak_labels.extend(rakshak[key][1])
    all_rakshak_labels = np.array(all_rakshak_labels)
    print("data shape: ", all_rakshak_data.shape)
    print("labels shape: ", all_rakshak_labels.shape)

data shape:  (2562, 48000)
labels shape:  (2562,)


In [20]:
for file in rakshak_split.keys():
    with open(file, 'w') as f:
        for item in rakshak_split[file]:
            f.write("%s\n" % item)
gc.collect()

0

In [21]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
if TRAIN == 'speech_cmd':
    speech_cmd['train'][0] = speech_cmd['train'][0].reshape(speech_cmd['train'][0].shape[0], 1, 16000 * 3)
    speech_cmd['test'][0] = speech_cmd['test'][0].reshape(speech_cmd['test'][0].shape[0], 1, 16000 * 3)
    speech_cmd['valid'][0] = speech_cmd['valid'][0].reshape(speech_cmd['valid'][0].shape[0], 1, 16000 * 3)
    rakshak['test'][0] = rakshak['test'][0].reshape(rakshak['test'][0].shape[0], 1, 16000 * 3)
    rakshak['valid'][0] = rakshak['valid'][0].reshape(rakshak['valid'][0].shape[0], 1, 16000 * 3)
    speech_cmd['train'][1] = labelencoder.fit_transform(speech_cmd['train'][1])
    speech_cmd['test'][1] = labelencoder.transform(speech_cmd['test'][1])
    speech_cmd['valid'][1] = labelencoder.transform(speech_cmd['valid'][1])
    rakshak['test'][1] = labelencoder.transform(rakshak['test'][1])
    rakshak['valid'][1] = labelencoder.transform(rakshak['valid'][1])
    mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))
    speech_cmd['train'][1] = to_categorical(speech_cmd['train'][1])
    speech_cmd['test'][1] = to_categorical(speech_cmd['test'][1])
    speech_cmd['valid'][1] = to_categorical(speech_cmd['valid'][1])
    rakshak['test'][1] = to_categorical(rakshak['test'][1])
    rakshak['valid'][1] = to_categorical(rakshak['valid'][1])
else:
    rakshak['train'][0] = rakshak['train'][0].reshape(rakshak['train'][0].shape[0], 1, 16000 * 3)
    rakshak['test'][0] = rakshak['test'][0].reshape(rakshak['test'][0].shape[0], 1, 16000 * 3)
    rakshak['valid'][0] = rakshak['valid'][0].reshape(rakshak['valid'][0].shape[0], 1, 16000 * 3)
    speech_cmd['test'][0] = speech_cmd['test'][0].reshape(speech_cmd['test'][0].shape[0], 1, 16000 * 3)
    speech_cmd['valid'][0] = speech_cmd['valid'][0].reshape(speech_cmd['valid'][0].shape[0], 1, 16000 * 3)
    rakshak['train'][1] = labelencoder.fit_transform(rakshak['train'][1])
    rakshak['test'][1] = labelencoder.transform(rakshak['test'][1])
    rakshak['valid'][1] = labelencoder.transform(rakshak['valid'][1])
    speech_cmd['test'][1] = labelencoder.transform(speech_cmd['test'][1])
    speech_cmd['valid'][1] = labelencoder.transform(speech_cmd['valid'][1])
    mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))
    rakshak['train'][1] = to_categorical(rakshak['train'][1])
    rakshak['test'][1] = to_categorical(rakshak['test'][1])
    rakshak['valid'][1] = to_categorical(rakshak['valid'][1])
    speech_cmd['test'][1] = to_categorical(speech_cmd['test'][1])
    speech_cmd['valid'][1] = to_categorical(speech_cmd['valid'][1])
    if TRAIN_RAKSHAK_ON_ALL_DATA:
        all_rakshak_data = all_rakshak_data.reshape(all_rakshak_data.shape[0], 1, 16000*3)
        all_rakshak_labels = labelencoder.transform(all_rakshak_labels)
        all_rakshak_labels = to_categorical(all_rakshak_labels)
print(mapping)

Using TensorFlow backend.


{'go': 0, 'no': 1, 'stop': 2, 'yes': 3}


In [22]:
inverted_mapping = dict(map(reversed, mapping.items()))
inverted_mapping

{0: 'go', 1: 'no', 2: 'stop', 3: 'yes'}

In [23]:
import tensorflow as tf
import kapre
from keras.models import Sequential
from keras.layers import Dense,AveragePooling2D
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise
import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D,SeparableConv2D,BatchNormalization,LSTM,Reshape,TimeDistributed


# 6 channels (!), maybe 1-sec audio signal, for an example.

sr = 16000
input_shape = (1,sr*3)


def edgespeechneta():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/esna")
    model = Sequential()
    # A mel-spectrogram layer
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
    # After this, it's just a usual keras workflow. For example..
    # Add some layers, e.g., model.add(some convolution layers..)
   
    # Compile the model
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(20, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(15, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(25, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(22, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(22, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(25, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(4, activation='softmax'))
    return model

def edgespeechnetb():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/edgespeechnet-b-100e")
    model = Sequential()
    # A mel-spectrogram layer
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
    # After this, it's just a usual keras workflow. For example..
    # Add some layers, e.g., model.add(some convolution layers..)
   
    # Compile the model
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(33, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(35, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))

    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(4, activation='softmax'))
    return model

def cnn():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/cnn-100e")
    model = Sequential()
    # A mel-spectrogram layer
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
    # After this, it's just a usual keras workflow. For example..
    # Add some layers, e.g., model.add(some convolution layers..)
    # Compile the model

    model.add(Conv2D(64, kernel_size=(20, 8), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(10, 4), activation='relu',dim_ordering="th"))
    model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation='softmax'))
    return model

model = edgespeechnetb()

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
trainable_stft (Melspectrogr (None, 128, 94, 1)        296064    
_________________________________________________________________
additive_noise_1 (AdditiveNo (None, 128, 94, 1)        0         
_________________________________________________________________
normalization2d_1 (Normaliza (None, 128, 94, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 92, 45)       450       
_________________________________________________________________
average_pooling2d_1 (Average (None, 63, 46, 45)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 44, 43)        17040     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 45, 42, 41)        12195     
__________

In [25]:
model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.adam(),metrics=['accuracy'],)
if TRAIN == 'speech_cmd':
    model.fit(speech_cmd['train'][0], speech_cmd['train'][1], batch_size=100, epochs=50, verbose=1, validation_data=(speech_cmd['valid'][0], speech_cmd['valid'][1]))
else:
    model.fit(all_rakshak_data, all_rakshak_labels, batch_size=50, epochs=50, verbose=1, validation_data=(rakshak['valid'][0], rakshak['valid'][1]))
model.save('rakshak/esnb_{}.h5'.format(TRAIN))
# model.fit(all_rakshak_data, all_rakshak_labels, batch_size=50, epochs=50, verbose=1, validation_data=(speech_cmd['valid'][0], speech_cmd['valid'][1]))

Train on 2562 samples, validate on 121 samples
Epoch 1/50
2562/2562 [==============================] - 5s 2ms/step - loss: 1.3306 - acc: 0.3458 - val_loss: 1.1769 - val_acc: 0.3719
Epoch 2/50
2562/2562 [==============================] - 2s 636us/step - loss: 1.1776 - acc: 0.4684 - val_loss: 1.0520 - val_acc: 0.5620
Epoch 3/50
2562/2562 [==============================] - 2s 634us/step - loss: 0.9718 - acc: 0.5648 - val_loss: 0.7803 - val_acc: 0.5950
Epoch 4/50
2562/2562 [==============================] - 2s 624us/step - loss: 0.7085 - acc: 0.6690 - val_loss: 0.7240 - val_acc: 0.7025
Epoch 5/50
2562/2562 [==============================] - 2s 616us/step - loss: 0.5461 - acc: 0.7521 - val_loss: 0.7889 - val_acc: 0.7107
Epoch 6/50
2562/2562 [==============================] - 2s 617us/step - loss: 0.4918 - acc: 0.7834 - val_loss: 0.6802 - val_acc: 0.7521
Epoch 7/50
2562/2562 [==============================] - 2s 620us/step - loss: 0.4021 - acc: 0.8372 - val_loss: 0.6726 - val_acc: 0.7934
Epo

In [26]:
def speech_cmd_test(which):
    colors = {'rakshak':'#ff7979', 'speech_cmd':'#f6e58d'}
    if 'speech_cmd' in TEST:
        from keras.models import load_model
        model1 = load_model('rakshak/esnb_{}.h5'.format(which), custom_objects={'Melspectrogram':kapre.time_frequency.Melspectrogram, 'AdditiveNoise':kapre.augmentation.AdditiveNoise,\
                                                 'Normalization2D':kapre.utils.Normalization2D})
        accuracy ={'valid':{}, 'test':{}}
        count = 0
        preds_valid = model1.predict(speech_cmd['valid'][0])
        preds_test = model1.predict(speech_cmd['test'][0])
        for i in range(preds_valid.shape[0]):
            if np.argmax(preds_valid[i]) == np.argmax(speech_cmd['valid'][1][i]):
                count += 1
                if inverted_mapping[np.argmax(preds_valid[i])] not in accuracy['valid'].keys():
    #                 print(inverted_mapping[np.argmax(preds_valid[i])])
                    accuracy['valid'][inverted_mapping[np.argmax(preds_valid[i])]] = 1
                else:
                    accuracy['valid'][inverted_mapping[np.argmax(preds_valid[i])]] += 1
        acc_valid = count/preds_valid.shape[0]
        print("accuracy on speech_cmd validation set: ", acc_valid)
        count = 0
        for i in range(preds_test.shape[0]):
            if np.argmax(preds_test[i]) == np.argmax(speech_cmd['test'][1][i]):
                count += 1
                if inverted_mapping[np.argmax(preds_test[i])] not in accuracy['test'].keys():
                    accuracy['test'][inverted_mapping[np.argmax(preds_test[i])]] = 1
                else:
                    accuracy['test'][inverted_mapping[np.argmax(preds_test[i])]] += 1
        acc_test = count/preds_test.shape[0]
        print("accuracy on speech_cmd test set: ", acc_test)
        idx = {'test':{}, 'valid':{}}
        for type_ in idx.keys():
            for file in speech_split[type_+'.txt']:
                key = file.split('/')[0]
                if key not in idx[type_].keys():
                    idx[type_][key] = 1
                else:
                    idx[type_][key] += 1
        total = {}
        for type_ in accuracy.keys():
            for key in accuracy[type_].keys():
                if key not in total.keys():
                    total[key] = accuracy[type_][key]
                else:
                    total[key] += accuracy[type_][key]
                accuracy[type_][key] = accuracy[type_][key]/idx[type_][key]
        for key in total.keys():
            total[key] = total[key]/(idx['test'][key]+idx['valid'][key])
        gc.collect()
        trace = go.Bar(
            x = [*total.keys()],
            y = [*total.values()],
            marker=dict(
                color=colors[which]
            ),
            name = which
        )
        return trace
    return None

In [27]:
def compare_plot(trace0, trace1):
    if (trace0 is None) or (trace1 is None):
        print("None data provided")
        return None
    else:
        data = [trace0, trace1]
        layout = go.Layout(
            title='accuracy on speech command',
        )

        fig = go.Figure(data=data, layout=layout)
        return py.iplot(fig, filename='accuracy_keywords_on_speech_cmd')

In [28]:
trace0 = speech_cmd_test('speech_cmd')
trace1 = speech_cmd_test('rakshak')

int_axis=1 passed but is ignored, str_axis is used instead.
accuracy on speech_cmd validation set:  0.9737024221453288
accuracy on speech_cmd test set:  0.9785615491009682
int_axis=1 passed but is ignored, str_axis is used instead.
accuracy on speech_cmd validation set:  0.6512110726643598
accuracy on speech_cmd test set:  0.656984785615491


### Model comparision on Speech Command

In [29]:
compare_plot(trace0, trace1)

### Rakshak place wise

In [30]:
count = 0
preds = model.predict(rakshak['valid'][0])
for i in range(preds.shape[0]):
    if np.argmax(preds[i]) == np.argmax(rakshak['valid'][1][i]):
        count += 1
print("accuracy: ", count/preds.shape[0])
idx = {}
length_dict = {}
for i,file in enumerate(rakshak_split['rakshak_valid.txt']):
    key = file.split('/')[0].lower()
    if key not in idx.keys():
        idx[key] = [i]
        length_dict[key] = 1
    else:
        idx[key].append(i)
        length_dict[key] += 1
print("number of samples of different places:\n", length_dict)
accuracy = {}
for key in idx.keys():
    count = 0
    for ids in idx[key]:
        if np.argmax(preds[ids]) == np.argmax(rakshak['valid'][1][ids]):
            count += 1
    accuracy[key] = count/len(idx[key])
print("accuracy based on locations:\n", accuracy)
with open(rakshak_path+TRAIN+'_placewise.txt', 'w') as f:
    for key in accuracy.keys():
        f.write("{}:{}\n".format(key, accuracy[key]))

accuracy:  0.8677685950413223
number of samples of different places:
 {'house': 79, 'india gate lake area ': 12, 'railway station': 4, 'quickspeech-in.appspot.com': 23, 'india gate': 3}
accuracy based on locations:
 {'house': 0.8987341772151899, 'india gate lake area ': 0.8333333333333334, 'railway station': 0.5, 'quickspeech-in.appspot.com': 0.8260869565217391, 'india gate': 1.0}


In [31]:
accuracy

{'house': 0.8987341772151899,
 'india gate lake area ': 0.8333333333333334,
 'railway station': 0.5,
 'quickspeech-in.appspot.com': 0.8260869565217391,
 'india gate': 1.0}

In [32]:
content_rakshak = {}
content_speech = {}
with open(rakshak_path+'rakshak_placewise.txt', 'r') as f:
    for line in f:
        content_rakshak[line.split(":")[0].strip()] = line.split(":")[1].strip()
with open(rakshak_path+'speech_cmd_placewise.txt', 'r') as f:
    for line in f:
        content_speech[line.split(":")[0].strip()] = line.split(":")[1].strip()

In [33]:
trace0 = go.Bar(
    x = [*content_rakshak.keys()],
    y = [*content_rakshak.values()],
    marker=dict(
        color='#ff7979'
    ),
    name = 'rakshak'
)

trace1 = go.Bar(
    x = [*content_speech.keys()],
    y = [*content_speech.values()],
    marker=dict(
        color='#f6e58d'
    ),
    name = 'speech_cmd'
)

data = [trace0, trace1]
layout = go.Layout(
    title='accuracy of keywords in different places',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='accuracy_keywords_rakshak')